<a href="https://colab.research.google.com/github/SaahilShaikh17/LoanDefaultPrediction/blob/main/LoanDefaultPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(readr)


In [3]:
url <- "https://github.com/SaahilShaikh17/LoanDefaultPrediction/blob/main/Training%20Data.csv"

df <- read.csv(url)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'https://github.com/SaahilShaikh17/LoanDefaultPrediction/blob/main/Training%20Data.csv'”
